In [17]:
import pandas as pd
import json
import geopandas as gp
from geopy import distance


df= pd.read_csv('sandbox-singlepage/data/housing_prepped.csv', dtype={"GEOID": str,"TRACT_NUM": str})

#filter for King County
df = df[(df['COUNTY'] == 'King')]

rdf = pd.read_csv('sandbox-singlepage/data/race-data.csv', dtype={"TRACT_NUM": str, "YEAR": str})

#filter for King County 2010
rdf = rdf[(rdf['COUNTY'] == 'King') & (rdf['YEAR'] == '2010')]

#create GEOID
rdf['GEOID'] = '53033' + rdf['TRACT_NUM']

gdf = pd.read_csv('sandbox-singlepage/data/washingtongeo_dist.csv',
                   dtype={"TRACTCE_a": str,"TRACTCE_b": str})

rdf.head(3)

,COUNTY,TRACT_NUM,DATA,CENSUS_QUERY,HUMAN_READABLE,DATASET,YEAR,GEOID
52326,King,000100,3852.0,B02001_002E,Estimate!!Total!!White alone-RACE,acs5,2010,53033000100
52335,King,000100,436.0,B02001_003E,Estimate!!Total!!Black or African American alo...,acs5,2010,53033000100
52344,King,000100,83.0,B02001_004E,Estimate!!Total!!American Indian and Alaska Na...,acs5,2010,53033000100


In [18]:
rdf.shape

(7164, 8)

In [20]:
gdf.shape#, gpydf.drop_duplicates().shape

(78606, 4)

In [21]:
white = rdf[(rdf['CENSUS_QUERY'] == 'B03002_003E')]
white = white[['GEOID','COUNTY','TRACT_NUM','DATA']]
white = white.rename(columns = {'DATA' : 'pop_white_nonhisp_only'})
black = rdf[(rdf['CENSUS_QUERY'] == 'B02001_003E')]
black = black[['GEOID','COUNTY','TRACT_NUM','DATA']]
black = black.rename(columns = {'DATA' : 'pop_black_only'})
native = rdf[(rdf['CENSUS_QUERY'] == 'B02001_004E')]
native = native[['GEOID','COUNTY','TRACT_NUM','DATA']]
native = native.rename(columns = {'DATA' : 'pop_native_only'})
asian = rdf[(rdf['CENSUS_QUERY'] == 'B02001_005E')]
asian = asian[['GEOID','COUNTY','TRACT_NUM','DATA']]
asian = asian.rename(columns = {'DATA' : 'pop_asian_only'})
polynesian = rdf[(rdf['CENSUS_QUERY'] == 'B02001_006E')]
polynesian = polynesian[['GEOID','COUNTY','TRACT_NUM','DATA']]
polynesian = polynesian.rename(columns = {'DATA' : 'pop_polynesian_only'})
latino = rdf[(rdf['CENSUS_QUERY'] == 'B03002_012E')]
latino = latino[['GEOID','COUNTY','TRACT_NUM','DATA']]
latino = latino.rename(columns = {'DATA' : 'pop_hispanic'})
other = rdf[(rdf['CENSUS_QUERY'] == 'B02001_007E')]
other = other[['GEOID','COUNTY','TRACT_NUM','DATA']]
other = other.rename(columns = {'DATA' : 'pop_other_only'})
multiracial = rdf[(rdf['CENSUS_QUERY'] == 'B02001_008E')]
multiracial = multiracial[['GEOID','COUNTY','TRACT_NUM','DATA']]
multiracial = multiracial.rename(columns = {'DATA' : 'pop_multiracial'})
racial = white.merge(black, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
racial = racial.merge(native, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
racial = racial.merge(asian, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
racial = racial.merge(polynesian, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
racial = racial.merge(latino, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
racial = racial.merge(other, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
racial = racial.merge(multiracial, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
race_data = racial[['GEOID','COUNTY','TRACT_NUM','pop_white_nonhisp_only','pop_black_only','pop_native_only','pop_asian_only','pop_polynesian_only','pop_hispanic','pop_other_only','pop_multiracial']]
race_data.head(2)
df = df.merge(race_data, how = 'inner', left_on = ['GEOID','COUNTY','TRACT_NUM'], right_on = ['GEOID','COUNTY','TRACT_NUM'])
df['minority_pop'] = df['TOT_POP_2010'] - df['pop_white_nonhisp_only']
df['minority_pop_pct'] = df['pop_white_nonhisp_only'] / df['TOT_POP_2010']

In [22]:
df.shape, df.drop_duplicates().shape

((398, 33), (398, 33))

In [23]:
df[['TRACT_NUM']].drop_duplicates().shape

(398, 1)

In [24]:
pd.set_option('display.max_columns', 500)
df[df['TRACT_NUM'] == '000401']
'index__muni'

'index__muni'

In [25]:
gdf = gdf.merge(df[['TRACT_NUM','GEOID']], how='left', left_on='TRACTCE_a', right_on='TRACT_NUM')
gdf = gdf.rename(columns={'GEOID':'GEOID_a'})
gdf.head(3)

,Unnamed: 0,TRACTCE_a,TRACTCE_b,distance,TRACT_NUM,GEOID_a
0,1,031705,032212,22236.625478,031705,53033031705
1,2,031705,032322,38423.760229,031705,53033031705
2,44,031705,032213,24613.040010,031705,53033031705


In [26]:
gdf.shape
gdf.drop_duplicates().shape

(78606, 6)

In [27]:
gdf = gdf.merge(df[['TRACT_NUM','GEOID']], how='left', left_on='TRACTCE_b', right_on='TRACT_NUM')
gdf = gdf.rename(columns={'GEOID':'GEOID_b'})
gdf.head(3)

,Unnamed: 0,TRACTCE_a,TRACTCE_b,distance,TRACT_NUM_x,GEOID_a,TRACT_NUM_y,GEOID_b
0,1,031705,032212,22236.625478,031705,53033031705,032212,53033032212
1,2,031705,032322,38423.760229,031705,53033031705,032322,53033032322
2,44,031705,032213,24613.040010,031705,53033031705,032213,53033032213


In [28]:
gdf.shape

(78606, 8)

In [29]:
#merge dataframes to combine the different datasets so that you can calculate it.
minority10 = df[['GEOID','minority_pop_pct']]
gdf = gdf.merge(minority10, how = 'inner', left_on = ['GEOID_a'], right_on = ['GEOID'])
gdf = gdf.rename(columns = {'minority_pop_pct':'minority_pop_pct_2010_a'})
gdf.head(3)

,Unnamed: 0,TRACTCE_a,TRACTCE_b,distance,TRACT_NUM_x,GEOID_a,TRACT_NUM_y,GEOID_b,GEOID,minority_pop_pct_2010_a
0,1,031705,032212,22236.625478,031705,53033031705,032212,53033032212,53033031705,0.850154
1,2,031705,032322,38423.760229,031705,53033031705,032322,53033032322,53033031705,0.850154
2,44,031705,032213,24613.040010,031705,53033031705,032213,53033032213,53033031705,0.850154


In [30]:
gdf = gdf.merge(minority10, how = 'inner', left_on = ['GEOID_b'], right_on = ['GEOID'])
gdf = gdf.rename(columns = {'minority_pop_pct':'minority_pop_pct_2010_b'})
gdf.head(3)

,Unnamed: 0,TRACTCE_a,TRACTCE_b,distance,TRACT_NUM_x,GEOID_a,TRACT_NUM_y,GEOID_b,GEOID_x,minority_pop_pct_2010_a,GEOID_y,minority_pop_pct_2010_b
0,1,031705,032212,22236.625478,031705,53033031705,032212,53033032212,53033031705,0.850154,53033032212,0.631176
1,1192,000200,032212,28735.234239,000200,53033000200,032212,53033032212,53033000200,0.700729,53033032212,0.631176
2,1589,000900,032212,25152.434293,000900,53033000900,032212,53033032212,53033000900,0.812346,53033032212,0.631176


In [31]:
df.head(3)


,COUNTY,TRACT_NUM,TOT_POP_2010,DATASET,GEOID,TOT_POP_2018,BORN_OTHER_US_STATE_2010,BORN_OTHER_US_STATE_2018,BORN_FOREIGN_75KPLUS_2010,BORN_FOREIGN_75KPLUS_2018,TWENTY_PCTILE_2010,TWENTY_PCTILE_2018,EIGHTY_PCTILE_2010,EIGHTY_PCTILE_2018,BORN_OTHER_US_STATE_PCT_2010,BORN_OTHER_US_STATE_PCT_2018,BORN_OTHER_US_STATE_PCT_CHANGE,BORN_FOREIGN_75KPLUS_PCT_2010,BORN_FOREIGN_75KPLUS_PCT_2018,BORN_FOREIGN_75KPLUS_PCT_CHANGE,INCOME_GAP_2010,INCOME_GAP_2018,INCOME_GAP_CHANGE,pop_white_nonhisp_only,pop_black_only,pop_native_only,pop_asian_only,pop_polynesian_only,pop_hispanic,pop_other_only,pop_multiracial,minority_pop,minority_pop_pct
0,King,000100,5784.0,acs5,53033000100,7985.0,362.0,453.0,211.0,238.0,17207.0,18832.0,99318.0,106103.0,6.258645,5.673137,-0.585507,3.647994,2.980589,-0.667406,17.325158,17.748791,0.423634,3330.0,436.0,83.0,710.0,11.0,988.0,412.0,280.0,2454.0,0.575726
1,King,000200,7682.0,acs5,53033000200,8498.0,315.0,664.0,71.0,317.0,20684.0,37303.0,100531.0,125836.0,4.100495,7.813603,3.713109,0.924238,3.730289,2.806051,20.574748,29.644140,9.069392,5383.0,181.0,32.0,1017.0,139.0,594.0,138.0,246.0,2299.0,0.700729
2,King,000300,2548.0,acs5,53033000300,2991.0,100.0,177.0,20.0,60.0,29271.0,41357.0,117517.0,152111.0,3.924647,5.917753,1.993106,0.784929,2.006018,1.221089,24.907886,27.188698,2.280812,1856.0,27.0,17.0,406.0,0.0,132.0,30.0,101.0,692.0,0.728414


In [32]:
gdf.head(3)

,Unnamed: 0,TRACTCE_a,TRACTCE_b,distance,TRACT_NUM_x,GEOID_a,TRACT_NUM_y,GEOID_b,GEOID_x,minority_pop_pct_2010_a,GEOID_y,minority_pop_pct_2010_b
0,1,031705,032212,22236.625478,031705,53033031705,032212,53033032212,53033031705,0.850154,53033032212,0.631176
1,1192,000200,032212,28735.234239,000200,53033000200,032212,53033032212,53033000200,0.700729,53033032212,0.631176
2,1589,000900,032212,25152.434293,000900,53033000900,032212,53033032212,53033000900,0.812346,53033032212,0.631176


In [33]:
df.shape

(398, 33)

In [34]:
gdf.shape

(78606, 12)